In [4]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
import csv
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
#신의 탑, 평범한 8반, 비질란테, 인생존망, 니편내편, 검은인간, 부활남
webtoon_df = pd.read_csv('전체플랫폼_keyword.csv')

In [2]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        #불용어사전 추가중
        with open('C:/Users/96ljc/Untitled Folder/레진/전체_sim7/stopwords.txt', 'r', encoding='utf-8') as f:
            list_file = f.readlines()
        self.stopwords = list_file[0].split(",")
        #self.stopwords =['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같',
        #                 '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', 
        #                 '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', 
        #                 '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', 
        #                 '월', '데', '자신', '안', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', 
        #                 '보이', '번', '나', '다른', '어떻', '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', 
        #                 '좀', '원', '잘', '통하', '소리', '놓', '레진', '연재', '공개', '독점', '인기', '수도', '없는', '제작', '작가',
        #                 '수상', '작', '수상작', '보고', '고', '해도', '통해', '위해', '귀염둥이', '관계']
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)      
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences

    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence)) 
                                       if noun not in self.stopwords and len(noun) > 1]))
        return nouns


class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

#TextRank
class Rank(object):
    def get_ranks(self, graph, d=0.85): 
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 
            link_sum = np.sum(A[:,id])
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) 
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        
        self.graph_matrix = GraphMatrix()
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)

        self.rank = Rank()
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    
    def keywords(self, word_num=5):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

        keywords = []
        index=[]
        
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)

        for idx in index:
            keywords.append(self.idx2word[idx])

        return keywords
# title_list=[]
# keyword_list=[]
# for i in total_df.index:
#     text = total_df.loc[i, '장르'] +" "+total_df.loc[i, '줄거리']
#     #print(text)
#     textrank = TextRank(text)
#     title_list.append(total_df.loc[i, '제목'])
#     keyword_list.append(textrank.keywords())

In [5]:
title_list=[]
keyword_list=[]
writer_list=[]
genre_list=[]
desc_list=[]
url_list=[]
img_list=[]
platform_list=[]
for i in webtoon_df.index:
    text = webtoon_df.loc[i, '장르'] +" "+webtoon_df.loc[i, '줄거리']
    #print(text)
    textrank = TextRank(text)
    title_list.append(webtoon_df.loc[i, '제목'])
    writer_list.append(webtoon_df.loc[i, '작가'])
    genre_list.append(webtoon_df.loc[i, '장르'])
    desc_list.append(webtoon_df.loc[i, '줄거리'])
    url_list.append(webtoon_df.loc[i, 'url'])
    img_list.append(webtoon_df.loc[i, '썸네일'])
    platform_list.append(webtoon_df.loc[i, '플랫폼'])
    keyword_list.append(textrank.keywords())

In [7]:
i=5994
con = webtoon_df.loc[i]
con_text = webtoon_df.loc[i, '장르'] + " " +webtoon_df.loc[i, '줄거리']
textrank=TextRank(con_text)
print(con)
print(con_text)
print(textrank.keywords())

Unnamed: 0                                                 1022
제목                                                          12월
작가                                                     강소소, 송래현
장르                                                          드라마
줄거리                    송래현 작가의 '12월' 전작보다 더 가슴아픈 사랑의 이야기가 펼쳐집니다
url                    https://webtoon.kakao.com/content/12월/82
썸네일           https://kr-a.kakaopagecdn.com/P/C/82/sharing/2...
플랫폼                                                       카카오웹툰
키워드                                        ['드라마', '사랑', '이야기']
Name: 5994, dtype: object
드라마 송래현 작가의 '12월' 전작보다 더 가슴아픈 사랑의 이야기가 펼쳐집니다
['드라마', '사랑']


In [8]:
web_data = pd.DataFrame()
web_data['제목'] = title_list

web_data['작가']=writer_list
web_data['장르']=genre_list
web_data['줄거리']=desc_list
web_data['url']=url_list
web_data['썸네일']=img_list
web_data['플랫폼']=platform_list

web_data['키워드'] = keyword_list
web_data.to_csv('통합_keyword.csv', encoding='utf-8-sig')

In [9]:
data = pd.read_csv('통합_keyword.csv', low_memory=False)
data.head(10)

,Unnamed: 0,제목,작가,장르,줄거리,url,썸네일,플랫폼,키워드
0,0,참교육,채용택 / 한가람,"스토리, 액션",무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n&lt;부...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['부활', '신석기', '채용', '한가람', '교육']"
1,1,신의 탑,SIU,"스토리, 판타지",자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년\n그리고 그런 소년을 시험하는 탑,https://comic.naver.com/webtoon/list?titleId=1...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['소녀', '스토리', '시험', '소년', '판타지']"
2,2,뷰티풀 군바리,설이 / 윤성원,"스토리, 드라마",'여자도 군대에 간다면?'본격 여자도 군대 가는 만화!,https://comic.naver.com/webtoon/list?titleId=6...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['군대', '만화', '스토리', '드라마', '여자도']"
3,3,윈드브레이커,조용석,"스토리, 스포츠",혼자서 자전거를 즐겨타던 모범생 조자현.\n원치 않게 자전거 크루의 일에 자꾸 휘말...,https://comic.naver.com/webtoon/list?titleId=6...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['모범생', '스트릿', '드라마', '스토리', '스포츠']"
4,4,퀘스트지상주의,박태준 만화회사,"스토리, 드라마","[외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하는 작품!\n공부, 싸...",https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['퀘스트', '고등학생', '공부', '공유', '드라마']"
5,5,쇼미더럭키짱!,박태준 / 김성모,"에피소드, 액션",고작 18살 나이로 부산을 꿇린 남자 강건마\n메마른 그의 가슴을 송두리째 불태울 ...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['남자', '래퍼', '사나이', '액션', '에피소드']"
6,6,장씨세가 호위무사,김인호 / 조형근,"스토리, 액션",‘당신이 부른 것이오. 나란 사람을... ’\n은둔고수 광휘. 호위무사 되다.\n웹...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['무협', '시대극', '고수', '무사', '웹소설']"
7,7,소녀의 세계,모랑지,"스토리, 드라마",완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나\n여러 갈등을 함께 ...,https://comic.naver.com/webtoon/list?titleId=6...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['드라마', '백조', '갈등', '맘씨', '소녀']"
8,8,백수세끼,치즈,"스토리, 드라마",백수 시절 내 곁을 지켜줬던 그녀... 돌아와 주면 안 되겠니?\n음식 메뉴마다 담...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['드라마', '메뉴', '백수', '스토리', '연애']"
9,9,유미의 작가 수칙,이동건,"스토리, 일상",본 컨텐츠는 이동건 작가가 전하는 네이버의 브랜드 웹툰입니다.\n프로 작가의 삶을 ...,https://comic.naver.com/webtoon/list?titleId=7...,https://shared-comic.pstatic.net/thumb/webtoon...,네이버 웹툰,"['브랜드', '스토리', '일상', '컨텐츠', '고민']"


In [41]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pandas import DataFrame
from multipledispatch import dispatch

data = pd.read_csv('통합_keyword.csv', low_memory=False)
row=[]
#print(df.tail(5))
@dispatch(str)
def get_key(str1):
    one = data.index[data['제목'] == str1]
    key = data.iloc[one[0], 8]
    #print(data.iloc[data.index[data['제목'] == str1], 8])
    print(str1)
    textrank = TextRank(key)
    row = [len(data),'추천키워드','작가','장르','줄거리','url','썸네일','플랫폼',str(textrank.keywords())]
    df = data.append(pd.Series(row, index=data.columns),ignore_index=True)
    df.iloc[-1] = row
    print(row[8])
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['키워드'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    index = len(df)-1
    sim = sorted(list(enumerate(cosine_sim[index])), key=lambda x: x[1], reverse=True)
    web_indices = [index[0] for index in sim[2:8]]
    print(df['제목'].iloc[web_indices])
    print(str(cosine_sim[index][web_indices]))
@dispatch(str, str)
def get_key(str1, str2):
    key=  data.iloc[data.index[data['제목'] == str1][0], 8] + data.iloc[data.index[data['제목'] == str2][0], 8]
    print(str1 + "\t" + str2)
    textrank = TextRank(key)
    text_key = textrank.keywords()
    row = [len(data), '추천키워드','작가','장르','줄거리','url','썸네일','플랫폼',str(text_key)]
    df = data.append(pd.Series(row, index=data.columns),ignore_index=True)
    df.iloc[-1] = row
    print(row[8])
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['키워드'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    index = len(df)-1
    sim = sorted(list(enumerate(cosine_sim[index])), key=lambda x: x[1], reverse=True)
    web_indices = [index[0] for index in sim[3:9]]
    print(df['제목'].iloc[web_indices])
    print(str(cosine_sim[index][web_indices]))
@dispatch(str, str, str)
def get_key(str1, str2, str3):
    key=  data.iloc[data.index[data['제목'] == str1][0], 8] + data.iloc[data.index[data['제목'] == str2][0], 8] + data.iloc[data.index[data['제목'] == str3][0], 8]
    print(str1 + "\t" + str2 + "\t" + str3)
    textrank = TextRank(key)
    text_key = textrank.keywords()
    row = [len(data), '추천키워드','작가','장르','줄거리','url','썸네일','플랫폼',str(text_key)]
    #row = [len(data), '추천키워드', str(text_key)]
    df = data.append(pd.Series(row, index=data.columns),ignore_index=True)
    df.iloc[-1] = row
    print(row[8])
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['키워드'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    index = len(df)-1
    sim = sorted(list(enumerate(cosine_sim[index])), key=lambda x: x[1], reverse=True)
    web_indices = [index[0] for index in sim[4:10]]
    print(df['제목'].iloc[web_indices])
    print(str(cosine_sim[index][web_indices]))
@dispatch(str, str, str, str)
def get_key(str1, str2, str3, str4):
    key=  data.iloc[data.index[data['제목'] == str1][0], 8] + data.iloc[data.index[data['제목'] == str2][0], 8] + data.iloc[data.index[data['제목'] == str3][0], 8] + data.iloc[data.index[data['제목'] == str4][0], 8]
    print(str1 + "\t" + str2 + "\t" + str3 + "\t" + str4)
    textrank = TextRank(key)
    text_key = textrank.keywords()
    row = [len(data), '추천키워드','작가','장르','줄거리','url','썸네일','플랫폼',str(text_key)]
    #row = [len(data), '추천키워드', str(text_key)]
    df = data.append(pd.Series(row, index=data.columns),ignore_index=True)
    df.iloc[-1] = row
    print(row[8])
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['키워드'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    index = len(df)-1
    sim = sorted(list(enumerate(cosine_sim[index])), key=lambda x: x[1], reverse=True)
    web_indices = [index[0] for index in sim[5:11]]
    print(df['제목'].iloc[web_indices])
    print(str(cosine_sim[index][web_indices]))
@dispatch(str, str, str, str, str)
def get_key(str1, str2, str3, str4, str5):
    key = data.iloc[data.index[data['제목'] == str1][0], 8] + data.iloc[data.index[data['제목'] == str2][0], 8] + data.iloc[data.index[data['제목'] == str3][0], 8] + data.iloc[data.index[data['제목'] == str4][0], 8] + data.iloc[data.index[data['제목'] == str5][0], 8]
    print(str1 + "\t" + str2 + "\t" + str3 + "\t" + str4 + "\t" + str5)
    textrank = TextRank(key)
    text_key = textrank.keywords()
    row = [len(data), '추천키워드','작가','장르','줄거리','url','썸네일','플랫폼',str(text_key)]
    #row = [len(data), '추천키워드', str(text_key)]
    df = data.append(pd.Series(row, index=data.columns),ignore_index=True)
    df.iloc[-1] = row
    print(row[8])
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['키워드'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    index = len(df)-1
    sim = sorted(list(enumerate(cosine_sim[index])), key=lambda x: x[1], reverse=True)
    web_indices = [index[0] for index in sim[5:11]]
    print(df['제목'].iloc[web_indices])
    print(str(cosine_sim[index][web_indices]))

In [42]:
get_key('부마님 거기 있어줄래요')
print("\n")
get_key('방과 후 전쟁활동', '데드데이즈(DEAD DAYS)')
print("\n")
get_key('방과 후 전쟁활동', '데드데이즈(DEAD DAYS)','하이브 3')
print("\n")
get_key('방과 후 전쟁활동', '데드데이즈(DEAD DAYS)','하이브 3','개장수')
print("\n")
get_key('부마님 거기 있어줄래요', '공주님 마음대로!', '너와 사는 오늘', '우리 내일 이혼해요', 'N번째 연애')

부마님 거기 있어줄래요
['공주', '당나라', '사랑', '남자', '시대극']
2191    공주전쟁 [연재]
6952         공주전쟁
8187      유니크한 그녀
225          연애혁명
2332         천일야화
5333           여혜
Name: 제목, dtype: object
[0.29040308 0.29040308 0.28992228 0.27999852 0.26825437 0.26214608]


방과 후 전쟁활동	데드데이즈(DEAD DAYS)
['드라마', '바이러스', '생존', '스릴러', '스토리']
871         의도적 외면
1433    심연의 하늘 시즌4
1049            개미
631         사람의 조각
86             신도림
2443       정해진 첫사랑
Name: 제목, dtype: object
[0.49092318 0.48192445 0.4818589  0.43274684 0.4147308  0.39273807]


방과 후 전쟁활동	데드데이즈(DEAD DAYS)	하이브 3
['곤충', '드라마', '바이러스', '전쟁', '생존']
2443    정해진 첫사랑
1211        개장수
236        정글쥬스
4018     스킵과 로퍼
86          신도림
5312    리턴 서바이벌
Name: 제목, dtype: object
[0.31611797 0.31422747 0.30547959 0.2889469  0.26522074 0.25215486]


방과 후 전쟁활동	데드데이즈(DEAD DAYS)	하이브 3	개장수
['곤충', '바이러스', '생존', '수가', '스릴러']
871          의도적 외면
1433     심연의 하늘 시즌4
1486        하이브 1~2
1792              연
6282             파동
475     어느날 갑자기 서울은
Name: 제목, dtype: object
[0.325773